# Código 04: Triângulos

## Código pré loop principal

### Inicialização do glfw e criação da janela

In [ ]:
#Bibliotecas
!pip install glfw
import glfw
!pip install pyopengl
from OpenGL.GL import *
import OpenGL.GL.shaders #Não é redundante?
!pip install numpy
import numpy as np

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(720, 600, "Triangulos", None, None)
glfw.make_context_current(window)

### Eventos de teclado e mouse

In [ ]:
def key_event(window,key,scancode,action,mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
    
glfw.set_mouse_button_callback(window,mouse_event)

### Shaders: código, espaço, compilação e linkagem

In [ ]:
#GLSL para Vertex Shader
vertex_code = """
        attribute vec2 position;
        void main(){
            gl_Position = vec4(position,0.0,1.0);
        }
        """
#GLSL para Fragment Shader
fragment_code = """
        void main(){
            gl_FragColor = vec4(0.0, 0.0, 0.0, 1.0);
        }
        """

#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Criação dos vértices

In [ ]:
#Criando espaço
vertices = np.zeros(10, [("position", np.float32, 2)])

#Preenchendo as coordenadas
vertices['position'] = [
                        ( 0.0, 1.0),
                        (-0.4, 0.4),
                        (-1.0, 0.4),
                        (-0.6,-0.2),
                        (-0.8,-1.0),
                        ( 0.0,-0.6),
                        ( 0.8,-1.0),
                        ( 0.6,-0.2),
                        ( 1.0, 0.4),
                        ( 0.4, 0.4)
                        ]

### Manipulação dos espaços de dados

In [ ]:
#Requisitando espaço de buffer para GPU
buffer = glGenBuffers(1)
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer)
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
#glBindBuffer(GL_ARRAY_BUFFER, buffer)

#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

### Exibindo na tela

In [ ]:
glfw.show_window(window)

## Loop principal

* `GL_TRIANGLES`: cria triângulo por triângulo
* `GL_TRIANGLE_STRIP`: cria vários triângulos "grudados". Lógica: esquece o primeiro ponto e utiliza o próximo para formar o triângulo
* `glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)`: define o modo de renderização para os polígonos tanto na frente (lado visível) quanto no verso (lado de trás) dos objetos.
    * `GL_FRONT_AND_BACK`: configuração se aplica tanto aos polígonos da frente quanto aos de trás
    * `GL_LINE`: polígonos serão renderizados apenas como linhas, em vez de serem preenchidos com cores ou texturas.

In [ ]:
while not glfw.window_should_close(window):
    glfw.poll_events()

    glPolygonMode(GL_FRONT_AND_BACK,GL_LINE) ## ative esse comando para enxergar os triângulos
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    
    #Primeiro número: vértice inicial
    #Segundo número: quantidade de vértices
    glDrawArrays(GL_TRIANGLE_STRIP, 0, 10)

    glfw.swap_buffers(window)

glfw.terminate()